# Teacher Queries

In [1]:
from data201 import db_connection, df_query
from pandas import DataFrame
conn = db_connection('sheql.ini')
cursor = conn.cursor()

### Dataframe visualization for testing purposes

In [2]:
def display_results(cursor):
    """
    If there are results in the cursor from a call to a
    stored procedure, display the results in a dataframe. 
    """
    columns = result.description
    
    if columns == None:
        return 
    
    else:
        column_names = [column_info[0] 
                        for column_info in columns]

        rows = cursor.fetchall()
    
        if len(rows) > 0:
            df = DataFrame(rows)
            df.columns = column_names
            
            display(df)
        
        else:
            return

## Get teacher's information
**INPUT**: User ID (immediately gets called upon logging in as a teacher)

In [3]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_information')

cursor.execute(
    """
    CREATE PROCEDURE teacher_information(
        IN uid INT
    )
    BEGIN
        SELECT 
            t.teacher_id AS "Teacher ID",
            CONCAT(t.first_name, " ", t.last_name) AS "Teacher Name",
            COUNT(DISTINCT c.name) AS "Courses Taught",
            COUNT(c.name) AS "Total Class Sessions Per Week"
        FROM users u
        JOIN teacher t USING (user_id)
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN school ON teaches.school_id = school.school_id
        WHERE t.user_id = uid
        GROUP BY t.teacher_id,
            "Teacher Name",
            "Number of Classes";
    END
    """
)

In [4]:
uid = 4
cursor.callproc('teacher_information', (uid,))

for result in cursor.stored_results():
    display_results(result)


,Teacher ID,Teacher Name,Courses Taught,Total Class Sessions Per Week
0,4,Robert Robinson,4,25


## Get all classes of a teacher
**INPUT**: Teacher ID

In [5]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_all_classes')

cursor.execute(
    """
    CREATE PROCEDURE teacher_all_classes(
        IN tid INT
    )
    BEGIN
        SELECT 
            t.teacher_id AS "Teacher ID",
            c.course_id AS "Course ID",
            c.name AS "Course Name",
            teaches.grade_level AS "Grade Level",
            school.name AS "School"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN school ON teaches.school_id = school.school_id
        WHERE teaches.teacher_id = tid
        GROUP BY t.teacher_id, 
            c.course_id,
            c.name,
            teaches.grade_level,
            school.name
        ORDER BY c.course_id;
    END
    """
)

In [6]:
tid = 14

cursor.callproc('teacher_all_classes', (tid,))

for result in cursor.stored_results():
    display_results(result)


,Teacher ID,Course ID,Course Name,Grade Level,School
0,14,2,Science,7,Powell Middle School
1,14,8,Computers,7,Powell Middle School


## Get a teacher's full course schedule
**INPUT**: Teacher ID 

In [7]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_course_schedule')

cursor.execute(
    """
    CREATE PROCEDURE teacher_course_schedule(
        IN tid INT
    )
    BEGIN
        SELECT 
            c.course_id AS "Course ID",
            c.name AS "Course Name",
            teaches.grade_level AS "Grade",
        GROUP_CONCAT(
            DISTINCT CONCAT(takes.start_time, "-", takes.end_time)
            ORDER BY takes.start_time
            SEPARATOR '\n') AS "Class Times",
        GROUP_CONCAT(
          DISTINCT CASE takes.day
            WHEN "Monday" THEN "M"
            WHEN "Tuesday" THEN "Tu"
            WHEN "Wednesday" THEN "W"
            WHEN "Thursday" THEN "Th"
            WHEN "Friday" THEN "F"
          END
        ORDER BY FIELD(takes.day, "Monday", "Tuesday", "Wednesday", "Thursday", "Friday")
        SEPARATOR " ") AS "Days"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN takes USING (course_id)
        JOIN course c USING (course_id)
        WHERE teaches.teacher_id = tid
            AND teaches.start_time = takes.start_time
            AND teaches.end_time = takes.end_time
        GROUP BY c.course_id,
            c.name,
            teaches.grade_level,
            takes.start_time,
            takes.end_time
        ORDER BY c.course_id;
    END
    """
)

In [8]:
tid = 4

cursor.callproc('teacher_course_schedule', (tid,))

for result in cursor.stored_results():
    display_results(result)


,Course ID,Course Name,Grade,Class Times,Days
0,2,Science,9,10:20:00-11:05:00,M Tu W Th F
1,3,English,11,11:15:00-12:00:00,M Tu W Th F
2,4,History,11,14:00:00-14:45:00,M Tu W Th F
3,6,PE,11,09:25:00-10:10:00,M Tu W Th F
4,6,PE,9,13:00:00-13:50:00,M Tu W Th F


## Get all students in one class of the teacher
**INPUT**: Teacher ID, course ID, and grade

In [9]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_class_all_students')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_class_all_students(
        IN tid INT,
        IN cid INT,
        IN grade INT
    )
    BEGIN
        SELECT 
            t.teacher_id AS "Teacher ID",
            s.student_id AS "Student ID",
            s.school_id AS "School ID",
            s.user_id AS "Student User ID",
            CONCAT(s.first_name, " ", s.last_name) AS "Student Name",
            c.course_id AS "Course ID",
            c.name AS "Course Name",
            s.grade_level AS "Grade"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN takes USING (course_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = cid
            AND teaches.grade_level = s.grade_level
            AND teaches.school_id = s.school_id
            AND teaches.homeroom_id = s.homeroom_id
            AND s.grade_level = grade
        GROUP BY t.teacher_id, 
            s.student_id,
            s.user_id,
            s.first_name,
            s.last_name,
            c.course_id,
            c.name
        ORDER BY s.grade_level, s.last_name, s.first_name, s.student_id;
    END
    """
)

In [10]:
tid = 4
cid = 6
grade = 11

cursor.callproc('teacher_one_class_all_students', (tid, cid, grade,))

for result in cursor.stored_results():
    display_results(result)


,Teacher ID,Student ID,School ID,Student User ID,Student Name,Course ID,Course Name,Grade
0,4,1934,8,2134,Christie Black,6,PE,11
1,4,1920,8,2120,Michelle Burns,6,PE,11
2,4,1741,8,1941,Daniel Chen,6,PE,11
3,4,1526,8,1726,Lisa Cortez,6,PE,11
4,4,1750,8,1950,Wendy Garrett,6,PE,11
5,4,1476,8,1676,Louis Hayes,6,PE,11
6,4,1904,8,2104,Patricia Hill,6,PE,11
7,4,1783,8,1983,Angela Hurley,6,PE,11
8,4,1964,8,2164,Eddie Johnson,6,PE,11
9,4,1678,8,1878,Robert Johnson,6,PE,11


## Get count of all students in one class of the teacher
**INPUT**: Teacher ID, course ID, and grade

In [11]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_class_student_count')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_class_student_count(
        IN tid INT,
        IN cid INT,
        IN grade INT
    )
    BEGIN
        SELECT 
            COUNT(DISTINCT s.student_id) AS "Student Count"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN takes USING (course_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = cid
            AND teaches.grade_level = s.grade_level
            AND teaches.school_id = s.school_id
            AND teaches.homeroom_id = s.homeroom_id
            AND s.grade_level = grade;
    END
    """
)

In [12]:
tid = 4
cid = 6
grade = 11

cursor.callproc('teacher_one_class_student_count', (tid, cid, grade,))

for result in cursor.stored_results():
    display_results(result)


,Student Count
0,25


## Get all students of all classes of a teacher
**INPUT**: Teacher ID 

In [13]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_all_students')

cursor.execute(
    """
    CREATE PROCEDURE teacher_all_students(
        IN tid INT
    )
   BEGIN
        SELECT 
            t.teacher_id AS "Teacher ID",
            s.student_id AS "Student ID",
            s.user_id AS "Student User ID",
            s.school_id AS "School ID",
            CONCAT(s.first_name, " ", s.last_name) AS "Student Name",
            c.course_id AS "Course ID",
            c.name AS "Course Name",
            s.grade_level AS "Grade_Level"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN takes USING (course_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
        GROUP BY t.teacher_id, 
            s.student_id,
            s.user_id,
            s.first_name,
            s.last_name,
            c.course_id,
            c.name
        ORDER BY c.course_id, s.grade_level, s.last_name, s.first_name, s.student_id;
    END
    """
)

In [14]:
tid = 4

cursor.callproc('teacher_all_students', (tid,))

for result in cursor.stored_results():
    display_results(result)


,Teacher ID,Student ID,Student User ID,School ID,Student Name,Course ID,Course Name,Grade_Level
0,4,939,1139,8,William Adkins,2,Science,9
1,4,1361,1561,8,Jessica Anderson,2,Science,9
2,4,1345,1545,8,Kelly Boyd,2,Science,9
3,4,1299,1499,8,Christopher Briggs,2,Science,9
4,4,1156,1356,8,Stephen Buchanan,2,Science,9
...,...,...,...,...,...,...,...,...
118,4,1047,1247,8,Derrick Wilkerson,6,PE,9
119,4,1166,1366,8,Charlotte Williams,6,PE,9
120,4,1346,1546,8,Jacqueline Williams,6,PE,9
121,4,1377,1577,8,Marcus Wilson,6,PE,9


## Get one student's schedule of classes that the teacher teaches
**INPUT**: Teacher ID and student ID

**When to use**: A teacher wants to see a student's entire class schedule (of classes the teacher teaches)

In [15]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_student_schedule')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_student_schedule(
        IN tid INT,
        IN sid INT
    )
    BEGIN
        SELECT 
            s.student_id AS "Student ID",
            CONCAT(s.first_name, " ", s.last_name) AS "Student Name",
            c.course_id AS "Course ID",
            s.school_id AS "School ID",
            c.name AS "Course Name",
            CONCAT(t.first_name, " ", t.last_name) AS "Teacher Name",
            CONCAT(takes.start_time, "-", takes.end_time) AS "Schedule",
            takes.day AS "Day"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN takes USING (course_id)
        JOIN course c USING (course_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
            AND s.student_id = sid
        GROUP BY s.student_id,
            s.first_name,
            s.last_name,
            c.course_id,
            c.name,
            t.first_name,
            t.last_name,
            takes.start_time,
            takes.end_time,
            takes.day
        ORDER BY c.course_id;
    END
    """
)

In [16]:
tid = 4
sid = 2053

cursor.callproc('teacher_one_student_schedule', (tid, sid,))

for result in cursor.stored_results():
    display_results(result)


## Get all of one student's grades in all classes the teacher teaches
**INPUT**: Teacher ID and student ID

**When to use**: A teacher wants to look at one student's grades in all classes the teacher teaches (to observe overall performance of a student under a teacher IF they take more than one class from that teacher)

In [17]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_student_all_grades')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_student_all_grades(
        IN tid INT,
        IN sid INT
    )
    BEGIN
        SELECT 
            s.student_id AS "Student ID",
            CONCAT(s.first_name, " ", s.last_name) AS "Student Name",
            c.course_id AS "Course ID",
            c.name AS "Course Name",
            gd.grade_type AS "Assignment",
            gd.score AS "Score",
            CASE
                WHEN gd.score >= 90 THEN "A"
                WHEN gd.score >= 80 THEN "B"
                WHEN gd.score >= 70 THEN "C"
                WHEN gd.score >= 60 THEN "D"
                ELSE "F"
            END AS "Letter Grade"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN grade_details gd USING (course_id)
        JOIN takes USING (student_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
            AND s.student_id = sid
        GROUP BY s.student_id,
            s.first_name,
            s.last_name,
            c.course_id,
            c.name,
            gd.grade_type,
            gd.score
        ORDER BY c.course_id;
    END
    """
)

In [18]:
tid = 4
sid = 2053

cursor.callproc('teacher_one_student_all_grades', (tid, sid,))

for result in cursor.stored_results():
    display_results(result)

## Get all of one student's grades in **one** class the teacher teaches
**INPUT**: Teacher ID, student ID and course ID

In [19]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_student_one_class_grades')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_student_one_class_grades(
        IN tid INT,
        IN sid INT,
        IN cid INT
    )
    BEGIN
        SELECT 
            gd.grade_type AS "Assignment",
            gd.score AS "Score",
            gd.weight AS "Weight",
            CASE
                WHEN gd.score >= 90 THEN "A"
                WHEN gd.score >= 80 THEN "B"
                WHEN gd.score >= 70 THEN "C"
                WHEN gd.score >= 60 THEN "D"
                ELSE "F"
            END AS "Letter Grade"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN grade_details gd USING (course_id)
        JOIN takes USING (student_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
            AND s.student_id = sid
            AND c.course_id = cid
        GROUP BY 
            gd.grade_type, 
            gd.score,
            gd.weight;
    END
    """
)

In [20]:
tid = 4
cid = 6

cursor.callproc('teacher_one_student_one_class_grades', (tid, sid, cid,))

for result in cursor.stored_results():
    display_results(result)

## Get the counts of each grade in each course
**INPUT**: Teacher ID and course ID

In [21]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_class_grade_counts')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_class_grade_counts(
        IN tid INT,
        IN cid INT,
        IN grade INT
    )
    BEGIN
        WITH student_grades AS (
        SELECT 
            s.student_id,
            ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) AS weighted_average
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN grade_details gd USING (course_id)
        JOIN takes USING (student_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
            AND c.course_id = cid
            AND teaches.grade_level = grade
        GROUP BY s.student_id
    ),
    letter_grades AS (
        SELECT 
            CASE 
                WHEN weighted_average >= 90 THEN 'A'
                WHEN weighted_average >= 80 THEN 'B'
                WHEN weighted_average >= 70 THEN 'C'
                WHEN weighted_average >= 60 THEN 'D'
                ELSE 'F'
            END AS letter_grade
        FROM student_grades
    )
    SELECT 
        letter_grade AS "Letter Grade",
        COUNT(*) AS "Grade Count"
    FROM letter_grades
    GROUP BY letter_grade
    ORDER BY letter_grade;
    END
    """
)

In [22]:
tid = 4
cid = 6
grade = 11

cursor.callproc('teacher_one_class_grade_counts', (tid, cid, grade))

for result in cursor.stored_results():
    display_results(result)

,Letter Grade,Grade Count
0,A,2
1,B,11
2,C,12


## Get student's calculated grade in one class
**INPUT**: Teacher ID, student ID and class ID

In [23]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_student_weighted_grade')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_student_weighted_grade(
        IN tid INT,
        IN sid INT,
        IN cid INT
    )
    BEGIN
        WITH student_grades AS (
        SELECT 
              ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) AS weighted_average
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN grade_details gd USING (course_id)
        JOIN takes USING (student_id)
        JOIN student s USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
            AND s.student_id = sid
            AND c.course_id = cid
        GROUP BY c.course_id, c.name
    )
    SELECT 
        weighted_average AS "Weighted Average",
        CASE 
            WHEN weighted_average >= 90 THEN 'A'
            WHEN weighted_average >= 80 THEN 'B'
            WHEN weighted_average >= 70 THEN 'C'
            WHEN weighted_average >= 60 THEN 'D'
            ELSE 'F'
        END AS "Letter Grade"
        FROM student_grades;
    END
    """
)

In [24]:
tid = 4
sid = 2053
cid = 6

cursor.callproc('teacher_one_student_weighted_grade', (tid, sid, cid,))

for result in cursor.stored_results():
    display_results(result)

## Get all attendance dates for a class
**INPUT**: Teacher ID and course ID

In [25]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_class_attendance_dates')

cursor.execute(
    """
    CREATE PROCEDURE teacher_class_attendance_dates(
        IN tid INT,
        IN cid INT
    )
    BEGIN
        SELECT 
            a.date AS "Date"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN takes USING (course_id)
        JOIN student s USING (student_id)
        JOIN attendance a USING (student_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = cid
            AND a.date = date
            AND teaches.grade_level = s.grade_level
            AND teaches.school_id = s.school_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.course_id = takes.course_id
        GROUP BY a.date
        ORDER BY a.date;
    END
    """
)

In [26]:
tid = 4
cid = 6

cursor.callproc('teacher_class_attendance_dates', (tid, cid,))

for result in cursor.stored_results():
    display_results(result)

,Date
0,2025-03-17
1,2025-03-18
2,2025-03-19
3,2025-03-20
4,2025-03-21
5,2025-03-25
6,2025-03-26
7,2025-03-27
8,2025-03-31
9,2025-04-01


## Get attendance of teacher's students on specific date
**INPUT**: Teacher ID and date

In [27]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_class_attendance_by_date')

cursor.execute(
    """
    CREATE PROCEDURE teacher_class_attendance_by_date(
        IN tid INT,
        IN date DATE
    )
    BEGIN
        SELECT DISTINCT
            s.student_id AS "Student ID",
            s.last_name AS "Last Name",
            s.first_name AS "First Name",
            a.status AS "Status",
            a.notes AS "Notes"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN takes USING (course_id)
        JOIN student s USING (student_id)
        JOIN attendance a USING (student_id)
        WHERE teaches.teacher_id = tid
            AND a.date = date
            AND teaches.grade_level = s.grade_level
            AND teaches.school_id = s.school_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.course_id = takes.course_id
        GROUP BY t.teacher_id, 
            s.student_id,
            s.first_name,
            s.last_name,
            c.course_id,
            a.date,
            a.status,
            a.notes
        ORDER BY s.last_name, s.first_name, s.student_id;
    END
    """
)

In [28]:
tid = 5
date = "2025-05-16"

cursor.callproc('teacher_class_attendance_by_date', (tid, date,))

for result in cursor.stored_results():
    display_results(result)

,Student ID,Last Name,First Name,Status,Notes
0,2201,Adams,Elizabeth,absent,doctor appointment
1,1787,Blackburn,Kevin,late,None
2,1532,Brown,Ashley,present,None
3,81,Brown,Crystal,late,None
4,2220,Burns,Brittany,late,None
5,1379,Butler,Ashley,present,None
6,313,Butler,Jenna,absent,excused absence
7,2175,Campbell,Jeffrey,present,None
8,1653,Cowan,Gregory,late,None
9,932,Cruz,Brittany,present,None


## Get attendance counts of teacher's students on specific date
**INPUT**: Teacher ID and date

In [29]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_attendance_counts_by_date')

cursor.execute(
    """
    CREATE PROCEDURE teacher_attendance_counts_by_date(
        IN tid INT,
        IN date DATE
    )
    BEGIN
        SELECT
            a.status AS "Status",
            COUNT(DISTINCT a.student_id) AS "Attendance Count"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN takes USING (course_id)
        JOIN student s USING (student_id)
        JOIN attendance a USING (student_id)
        WHERE teaches.teacher_id = tid
            AND a.date = date
            AND teaches.grade_level = s.grade_level
            AND teaches.school_id = s.school_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.course_id = takes.course_id
        GROUP BY a.status
        ORDER BY a.status;
    END
    """
)

In [30]:
tid = 4
date = "2025-03-17"

cursor.callproc('teacher_attendance_counts_by_date', (tid, date,))

for result in cursor.stored_results():
    display_results(result)

,Status,Attendance Count
0,present,68
1,absent,3
2,late,1
3,excused,1


## Get all contact information of all guardians in all classes
**INPUT**: Teacher ID

**When to use**: A teacher wants to contact ALL guardians of students in their classes

In [31]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_all_classes_all_guardians')

cursor.execute(
    """
    CREATE PROCEDURE teacher_all_classes_all_guardians(
        IN tid INT
    )
    BEGIN
        SELECT 
            t.teacher_id AS "Teacher ID",
            s.student_id AS "Student ID",
            CONCAT(s.first_name, " ", s.last_name) AS "Student Name",
            CONCAT(g.first_name, " ", g.last_name) AS "Guardian Name",
            gsr.relationship AS "Relationship to Student",
            g.phone_number AS "Guardian Phone Number"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN grade_details gd USING (course_id)
        JOIN takes USING (student_id)
        JOIN student s USING (student_id)
        JOIN guardian_student_relationship gsr USING (student_id)
        JOIN guardian g USING (guardian_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
        GROUP BY t.teacher_id, 
            s.student_id,
            s.first_name,
            s.last_name,
            g.first_name,
            g.last_name,
            gsr.relationship,
            g.phone_number
        ORDER BY s.last_name, s.first_name;
    END
    """
)

In [32]:
tid = 4

cursor.callproc('teacher_all_classes_all_guardians', (tid,))

for result in cursor.stored_results():
    display_results(result)

,Teacher ID,Student ID,Student Name,Guardian Name,Relationship to Student,Guardian Phone Number
0,4,939,William Adkins,Kimberly Adkins,father,(802) 593-1124
1,4,939,William Adkins,Hannah Garrett,mother,(934) 396-6830
2,4,2853,Bryan Alvarez,Linda Oliver,father,(608) 278-8424
3,4,2853,Bryan Alvarez,David Alvarez,mother,(436) 292-9984
4,4,1361,Jessica Anderson,Henry Allen,grandfather,(944) 690-4052
...,...,...,...,...,...,...
103,4,2658,Kevin Wood,Leah Rivas,father,(497) 672-5563
104,4,1784,Robert Wood,Timothy Phillips,others,(284) 252-5700
105,4,2990,Matthew Wright,Jacob Mitchell,father,(772) 827-7275
106,4,2671,Diane Yates,Wendy Ramirez,grandmother,(902) 881-7862


## Get all contact information of one class's guardians
**INPUT**: Teacher ID and course ID

**When to use**: A teacher wants to contact the guardians of only one of their classes

In [33]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_class_all_guardians')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_class_all_guardians(
        IN tid INT,
        IN cid INT
    )
    BEGIN
        SELECT 
            t.teacher_id AS "Teacher ID",
            s.student_id AS "Student ID",
            c.course_id AS "Course ID",
            CONCAT(s.first_name, " ", s.last_name) AS "Student Name",
            CONCAT(g.first_name, " ", g.last_name) AS "Guardian Name",
            gsr.relationship AS "Relationship to Student",
            g.phone_number AS "Guardian Phone Number"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN grade_details gd USING (course_id)
        JOIN takes USING (student_id)
        JOIN student s USING (student_id)
        JOIN guardian_student_relationship gsr USING (student_id)
        JOIN guardian g USING (guardian_id)
        WHERE teaches.teacher_id = tid
            AND teaches.course_id = takes.course_id
            AND teaches.homeroom_id = s.homeroom_id
            AND teaches.school_id = s.school_id
            AND teaches.grade_level = s.grade_level
            AND c.course_id = cid
        GROUP BY t.teacher_id, 
            s.student_id, 
            c.course_id,
            s.first_name, 
            s.last_name, 
            g.first_name,
            g.last_name,
            gsr.relationship, 
            g.phone_number
        ORDER BY s.last_name, s.first_name;
    END
    """
)

In [34]:
tid = 4
cid = 6

cursor.callproc('teacher_one_class_all_guardians', (tid, cid,))

for result in cursor.stored_results():
    display_results(result)

,Teacher ID,Student ID,Course ID,Student Name,Guardian Name,Relationship to Student,Guardian Phone Number
0,4,939,6,William Adkins,Hannah Garrett,mother,(934) 396-6830
1,4,939,6,William Adkins,Kimberly Adkins,father,(802) 593-1124
2,4,1361,6,Jessica Anderson,Henry Allen,grandfather,(944) 690-4052
3,4,1934,6,Christie Black,Danielle Black,mother,(250) 533-3381
4,4,1934,6,Christie Black,Scott Kelley,grandmother,(764) 825-3819
...,...,...,...,...,...,...,...
68,4,1346,6,Jacqueline Williams,Jennifer Williams,others,(694) 448-1916
69,4,1377,6,Marcus Wilson,Mary Wilson,grandfather,(668) 849-8450
70,4,1377,6,Marcus Wilson,Cindy Wilson,father,(284) 231-1137
71,4,1784,6,Robert Wood,Timothy Phillips,others,(284) 252-5700


## Get all contact information of one student's guardians
**INPUT**: Teacher ID and student ID

**When to use**: A teacher wants to contact the guardians of only one student in their class

In [35]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_one_student_all_guardians')

cursor.execute(
    """
    CREATE PROCEDURE teacher_one_student_all_guardians(
        IN tid INT,
        IN sid INT
    )
    
    BEGIN
        SELECT 
            g.first_name AS "First Name",
            g.last_name AS "Last Name",
            g.phone_number AS "Guardian Phone Number",
            u.email AS "Guardian Email",
            gsr.relationship AS "Relationship to Student"
        FROM teacher t
        JOIN teaches USING (teacher_id)
        JOIN course c USING (course_id)
        JOIN grade_details gd USING (course_id)
        JOIN student s USING (student_id)
        JOIN guardian_student_relationship gsr USING (student_id)
        JOIN guardian g USING (guardian_id)
        JOIN users u ON u.user_id = g.user_id
        WHERE t.teacher_id = tid
            AND s.grade_level = teaches.grade_level
            AND s.school_id = t.school_id
            AND s.student_id = sid
            AND u.user_id = g.user_id
        GROUP BY t.teacher_id,
            g.first_name,
            g.last_name,
            gsr.relationship,
            g.phone_number,
            u.email;
    END
    """
)

In [36]:
tid = 5
sid = 2981

cursor.callproc('teacher_one_student_all_guardians', (tid, sid,))

for result in cursor.stored_results():
    display_results(result)

,First Name,Last Name,Guardian Phone Number,Guardian Email,Relationship to Student
0,Marcus,Morrow,(365) 384-3188,gua_m.morrow2@dawson.com,father
1,Jennifer,Fischer,(308) 566-4079,gua_j.fischer@james.biz,grandfather


----

# Grade Modification

## Delete a grade
**INPUT**: Student ID, course ID, and grade type

In [37]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_delete_grade')

cursor.execute(
    """
    CREATE PROCEDURE teacher_delete_grade(
        IN student_id INT, 
        IN course_id INT, 
        IN grade_type VARCHAR(50)
    )
    BEGIN
        DELETE FROM grade_details gd
        WHERE gd.student_id = student_id
            AND gd.course_id = course_id
            AND gd.grade_type = grade_type;
    END
    """
)

In [38]:
# remove homework 1
student_id = 981
course_id = 1
grade_type = "homework1"

cursor.callproc('teacher_delete_grade', (student_id, course_id, grade_type,))

(981, 1, 'homework1')

In [39]:
# check if homework 1 was deleted
tid = 3
sid = 981
cid = 1

cursor.callproc('teacher_one_student_one_class_grades', (tid, sid, cid,))

for result in cursor.stored_results():
    display_results(result)

,Assignment,Score,Weight,Letter Grade
0,homework2,88,0.05,B
1,quiz,74,0.20,C
2,mid exam,85,0.30,B
3,final exam,36,0.40,F


## Add a grade
**INPUT**: Student ID, course ID, grade type, score and weight

In [40]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_add_grade')

cursor.execute(
    """
    CREATE PROCEDURE teacher_add_grade(
        IN student_id INT, 
        IN course_id INT, 
        IN grade_type VARCHAR(50),
        IN score INT,
        IN weight DECIMAL(3, 2)
    )
    BEGIN
        INSERT INTO grade_details (student_id, course_id, grade_type, score, weight)
        VALUES (student_id, course_id, grade_type, score, weight);
    END
    """
)

In [41]:
# add homework 1
student_id = 981
course_id = 1
grade_type = "homework1"
score = 92
weight = .02

cursor.callproc('teacher_add_grade', (student_id, course_id, grade_type, score, weight,))

(981, 1, 'homework1', 92, Decimal('0.02'))

In [42]:
# check if added
tid = 3
sid = 981
cid = 1

cursor.callproc('teacher_one_student_one_class_grades', (tid, sid, cid,))

for result in cursor.stored_results():
    display_results(result)

,Assignment,Score,Weight,Letter Grade
0,homework1,92,0.02,A
1,homework2,88,0.05,B
2,quiz,74,0.20,C
3,mid exam,85,0.30,B
4,final exam,36,0.40,F


## Update a grade
**INPUT**: Student ID, course ID, grade type and score

In [43]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_update_grade')

cursor.execute(
    """
    CREATE PROCEDURE teacher_update_grade(
        IN student_id INT, 
        IN course_id INT, 
        IN grade_type VARCHAR(50),
        IN score INT
    )
    BEGIN
        UPDATE grade_details gd
        SET gd.score = score
        WHERE gd.student_id = student_id
            AND gd.course_id = course_id
            AND gd.grade_type = grade_type;
    END
    """
)

In [44]:
# update grade of homework 1
student_id = 981
course_id = 1
grade_type = "homework1"
score = 85

cursor.callproc('teacher_update_grade', (student_id, course_id, grade_type, score,))

(981, 1, 'homework1', 85)

In [45]:
# check if updated
tid = 3
sid = 981
cid = 1

cursor.callproc('teacher_one_student_one_class_grades', (tid, sid, cid,))

for result in cursor.stored_results():
    display_results(result)

,Assignment,Score,Weight,Letter Grade
0,homework1,85,0.02,B
1,homework2,88,0.05,B
2,quiz,74,0.20,C
3,mid exam,85,0.30,B
4,final exam,36,0.40,F


----

# Attendance Recording

## Add today's attendance
**INPUT**: Student ID, date, status, teacher ID, notes

In [46]:
cursor.execute('DROP PROCEDURE IF EXISTS teacher_add_attendance')

cursor.execute(
    """
    CREATE PROCEDURE teacher_add_attendance(
        IN student_id INT, 
        IN date DATE, 
        IN status VARCHAR(50),
        IN teacher_id INT,
        IN notes VARCHAR(50)
    )
    BEGIN
        INSERT INTO attendance
        VALUES (student_id, date, status, teacher_id, notes);
    END
    """
)

In [47]:
# add attendance
student_id = 2053
date = "2025-05-29"
status = "present"
teacher_id = 4
notes = None

cursor.callproc('teacher_add_attendance', (student_id, date, status, teacher_id, notes,))

(2053, '2025-05-29', 'present', 4, None)

In [48]:
# check attendance
tid = 4
date = "2025-05-29"

cursor.callproc('teacher_class_attendance_by_date', (tid, date,))

for result in cursor.stored_results():
    display_results(result)

In [49]:
cursor.close()
conn.close()